In [18]:
from sklearn import cluster
from sklearn.metrics.pairwise import cosine_similarity, euclidean_distances

import pandas as pd
import numpy as np

import re
from time import sleep

import pickle
from pymongo import MongoClient

import matplotlib.pyplot as plt
%matplotlib inline

In [19]:
import pprint

def SaveDict(filename, mode, root):
    if filename[-4:]!=".txt":
        filename+=".txt"
    with open(filename, mode) as f:
        f.write(pprint.pformat(root))

def LoadDict(filename):
    if filename[-4:]!=".txt":
        filename+=".txt"
    with open(filename, "r") as f:
        return eval(f.read())

In [20]:
client = MongoClient()
db = client.twitter

In [28]:
def hellinger_dist(vec1, vec2):
    dist = np.sqrt(0.5 * ((np.sqrt(vec1) - np.sqrt(vec2))**2).sum())
    return dist

In [29]:
def to_vec(topic_dict):
    vec = []
    for topic in topic_dict:
        vec.append(topic.split(" ")[1], topic_dict[topic])
    return vec

In [48]:
def get_user_vec(name, n_topics):
    user = db.user_merge.find_one({"name": name})
    if user:
        topic_vec = np.zeros(n_topics)
        for i in range(n_topics):
            if "topic "+str(i) in user:
                topic_vec[i] = user["topic "+str(i)]
        return topic_vec
    else:
        return "user not found!"

In [49]:
group_topics = pickle.load(open('./data/group_lda_bow_topics.pkl','rb'))

In [50]:
def predict_user_group(name):
    dist = np.zeros(len(group_topics))
    for i in range(len(group_topics)):
        user_vec = get_user_vec(name, len(group_topics[i]))
        dist[i] = hellinger_sim(group_topics[i], user_vec)
    return np.argmin(dist)

In [51]:
accuracy = 0
count = 0
for user in db.active_users.find():
    pred_group = predict_user_group(user["name"])
    true_group = user["label"]
    accuracy += (pred_group==true_group)
    count += 1
print accuracy/float(count)

0.293089878797


In [ ]:
similarity = gensim.matutils.cossim(lda_vec1, lda_vec2)